In [ ]:
# --- Cell 1: Setup and Widget Display ---
# Run this cell first

# 1. Installs and Imports
!pip install selenium ipywidgets pytz beautifulsoup4 webdriver-manager -q # Ensure libraries are installed
import os
import tempfile
import time
import csv
import shutil
from selenium import webdriver
from selenium.common.exceptions import (
    TimeoutException, ElementClickInterceptedException, NoSuchElementException,
    StaleElementReferenceException
)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta, date
from pytz import timezone
from IPython.display import display, clear_output
import ipywidgets as widgets

print("--- Libraries imported ---")

# 2. Your Stock Data (Paste your full list here)
availableTags = [
    "786 - 786 Investments Ltd", "AABS - AL- Abbas Sugar Mills Ltd.", "AAL - Agro Allianz Ltd.(Defaulter)",
    "AASM - AL-Abid Silk Mills Ltd.(Defaulter)", "AATM - Ali Asghar Textile Mills Ltd.", "ABL - Allied Bank Ltd.",
    "ABOT - Abbott Laboratories (Pak) Ltd.", "ABSON - Abson Industries Ltd.(Defaulter)", "ACPL - Attock Cement Pakistan Ltd.",
    "ADAMS - Adam Sugar Mills Ltd.", "ADMM - Artistic Denim Mills Ltd.", "ADOS - Ados Pakistan Ltd. Delisted",
    "ADTM - Adil Textile Mills Ltd.", "AEL - AEL Textile Ltd. (Delisted)", "AGHA - Agha Steel Industries Ltd",
    "AGIC - Askari General Insurance Company. Ltd.", "AGIL - Agriautos Industries Ltd.", "AGL - Agritech Ltd.",
    "AGLNCPS - Agritech Non-Voting Pref Class A", "AGP - AGP Ltd.", "AGSML - Abdullah Shah Ghazi Sugar Mills Ltd.",
    "AGTL - AL-Ghazi Tractors Ltd.", "AHCL - Arif Habib Corporation Ltd. Consolidated", "AHL - Arif Habib Ltd.",
    "AHTM - Ahmad Hassan Textile Mills Ltd.", "AICL - Adamjee Insurance Company Ltd.", "AIRLINK - Air Link Communication Ltd",
    "AKBL - Askari Bank Ltd.", "AKDHL - AKD Hospitality Ltd.", "AKDSL - AKD Securities Ltd.(Formerly BIPL Securities)",
    "AKGL - AL- Khair Gadoon Ltd.", "AKZO - Akzo Nobel Pakistan (Delisted)", "ALAC - Askari Life Assurance Company Ltd.",
    "ALIFE - Adamjee Life Assurance Company Ltd.", "ALNRS - Al-Noor  Sugar Mills Ltd.", "ALQT - AL-Qadir Textile Mills Ltd.",
    "ALTN - Altern Energy Ltd.", "AMBL - Apna Microfinance Bank Ltd.", "AMSL - Al-Mal Securities & Services Ltd.(Defaulter).",
    "AMTEX - Amtex Ltd.(Defaulter)", "ANL - Azgard Nine Ltd.", "ANLNV - Azgard Nine Non-Voting Ordinary Shares",
    "ANLPS - Azgard Nine (Preference) 8.95%", "ANNT - Annoor Textile Mills Ltd.(Defaulter)", "ANSM - Ansari Sugar Mills Ltd.(Defaulter)",
    "ANTM - AN Textile Mills Ltd.", "APL - Attock Petroleum Ltd.", "APOT - Apollo Textile Mills Ltd.(Defaulter)",
    "AQTM - Al-Qaim Textile Mills Ltd.", "ARCTM - Arctic Textile Mills Ltd.", "ARM - Allied Rental Modaraba",
    "ARPAK - Arpak International Investment Ltd.", "ARPL - Archroma Pakistan Ltd.", "ARUJ - Aruj Industries Ltd. (Defaulter)",
    "ASC - Al Shaheer Corporation Ltd. (Defaulter)", "ASHT - Ashfaq Textile Mills Ltd.", "ASIC - Asia Insurance Company Ltd.",
    "ASL - Aisha Steel Mills Ltd.", "ASLCPS - Aisha Steel Convt Pref Right", "ASLPS - Aisha Steel Convertible Pref Shares",
    "ASTL - Amreli Steels Ltd.", "ASTM - Asim Textile Mills Ltd.", "ATBA - Atlas Battery Ltd.", "ATIL - Atlas Insurance Ltd.",
    "ATLH - Atlas Honda Ltd.", "ATRL - Attock Refinery Ltd.", "AVN - Avanceon Ltd. Consolidated",
    "AWTX - Allawasaya Textile & Finishing Mills Ltd.", "AWWAL - Awwal Modaraba (Delisting)", "AYTM - Ayesha Textile Mills Ltd.",
    "AYZT - Ayaz Textile Mills Ltd.", "AZMT - Azmat Textile Mills Ltd.(Defaulter)", "AZTM - Al-Azhar Textile Mills Ltd.",
    "BAFL - Bank Alfalah Ltd.", "BAFS - Baba Farid Sugar Mills Ltd", "BAHL - Bank AL-Habib Ltd.",
    "BAPL - Bawany Air Product Ltd.(Defaulter)", "BATA - Bata Pakistan Ltd.", "BBFL - Big Bird Foods Ltd",
    "BCL - Bolan Casting Ltd.", "BCML - Babri Cotton Mills Ltd. (Delisting)", "BECO - Beco Steel Ltd.(Formerly Ravi Textile)",
    "BEEM - Beema-Pakistan Company Ltd.", "BELA - Bela Automotives Ltd.(Defaulter)", "BERG - Berger Paints Pakistan Ltd.",
    "BFAGRO - Barkat Frisian Agro Ltd", "BFBIO - BF Biosciences Ltd", "BFMOD - B. F. Modaraba", "BGL - Baluchistan Glass Ltd.",
    "BHAT - Bhanero Textile Mills Ltd.", "BIFO - Biafo Industries Ltd.", "BIIC - Business & Industrial Insurance Company Ltd.(Defau",
    "BILF - Bilal Fibres Ltd.(Defaulter)", "BIPL - Bankislami Pakistan", "BML - Bank Makramah Ltd.(Summit Bank)",
    "BNL - Bunnys Ltd.", "BNWM - Bannu Woollen Mills Ltd.", "BOK - The Bank of Khyber", "BOP - The Bank Of Punjab",
    "BPBL - Baluchistan Particle Board Ltd.", "BPL - Burshane LPG (Pak) Ltd.", "BROT - Brothers Textile Mills Ltd.",
    "BRRG - B.R.R. Guardian Ltd.", "BTL - Blessed Textile Mills Ltd.", "BUXL - Buxly Paints Ltd.", "BWCL - Bestway Cement Ltd",
    "BWHL - Baluchistan Wheels Ltd.", "CASH - CALCORP Ltd.", "CCM - Crescent Cotton Mills Ltd.",
    "CECL - The Climax Engineering Co. Ltd.", "CENI - Century Insurance Company Ltd.", "CEPB - Century Paper & Board Mills Ltd.",
    "CFL - Crescent Fibres Ltd.", "CHAS - Chashma Sugar Mills Ltd.", "CHBL - Chenab Ltd.(Defaulter)",
    "CHCC - Cherat Cement Company Ltd.", "CJPL - Crescent Jute Proudcts Ltd.(Defaulter)", "CLCPS - Chenab Ltd.Pref.Sh",
    "CLOV - Clover Pakistan Ltd.", "CLVL - Cordoba Logistics & Ventures Ltd.", "CNERGY - Cnergyico Pk Ltd. (Byco Petroleum)",
    "COLG - Colgate Palmolive (Pak) Ltd.", "COST - (Colony) Sarhad Textile Mills Ltd.", "COTT - (Colony) Thal Textile Mills Ltd.",
    "CPHL - Citi Pharma Ltd.", "CPPL - Cherat Packaging Ltd.", "CRTM - Crescent Textile Mills Ltd.",
    "CSAP - Crescent Steel and Allied Proudcts Ltd.", "CSIL - Crescent Star Insurance Ltd.", "CSM - Crescent Standard Modaraba",
    "CTM - Colony Textile Mills Ltd.", "CWSM - Chakwal Spinning Mills Ltd.(Defaulter)", "CYAN - Cyan Ltd.",
    "DAAG - Data Agro Ltd.", "DADX - Dadex Eternit Ltd.", "DATM - Data Textiles Ltd. (Delisted)",
    "DBCI - Dadabhoy Cement Ind. Ltd.(Defaulter)", "DBSL - Dadabhoy Sack Ltd.(Defaulter)", "DCL - Dewan Cement Ltd.",
    "DCR - Dolmen City Reit", "DCTL - Dadabhoy Construction Tech. Ltd.(Defaulter)", "DEL - Dawood Equities Ltd.",
    "DFML - Dewan Farooque Motors Ltd.", "DFSM - Dewan Farooque Spinning Mills Ltd.", "DGKC - D. G. Khan Cement Company Ltd.",
    "DHPL - DH Partners Ltd", "DIIL - Diamond Industries Ltd.", "DINT - Din Textile Mills Ltd.",
    "DKTM - Dewan Khalid  Textile Mills Ltd.(Defaulter)", "DLL - Dawood Lawrencepur Ltd.",
    "DMTM - Dewan Mushtaq Textile Mills Ltd.(Defaulter)", "DMTX - D. M. Textile Mills Ltd.(Defaulter)",
    "DNCC - Dandot Cement Company Ltd.", "DOL - Descon Oxychem Ltd.", "DOMF - Dominion Stock Fund Ltd.(Defaulter)",
    "DSFL - Dewan Salman Fibre Ltd.(Defaulter)", "DSIL - D.S. Industries Ltd.", "DSL - Dost Steels Ltd.(Defaulter)",
    "DWAE - Dewan Automotive Engineering Ltd.(Defaulter)", "DWSM - Dewan Sugar Mills Ltd.(Defaulter)",
    "DWTM - Dewan Textile Mills Ltd.(Defaulter)", "DYNO - Dynea Pakistan Ltd.", "ECOP - Ecopack Ltd.",
    "EFERT - Engro Fertilizers Ltd.", "EFUG - EFU General Insurance Ltd.", "EFUL - EFU Life Assurance Ltd.",
    "ELCM - Elahi Cotton Mills Ltd.", "ELSM - Ellcot  Spinning Mills Ltd.", "EMCO - EMCO Industries Ltd.",
    "ENGL - English Leasing Ltd.(Defaulter)", "ENGRO - Engro Corporation Ltd. Consolidated",
    "ENGROH - Engro Holding Ltd (Formerly DAWH). Consolidated", "EPCL - Engro Polymer & Chemicals Ltd.",
    "EPQL - Engro Powergen Qadirpur Ltd.", "ESBL - Escorts Investment Bank Ltd.", "EWIC - East West Insurance Co. Ltd.",
    "EXIDE - Exide Pakistan Ltd.", "FABL - Faysal Bank Ltd.", "FAEL - Fatima Enterprises Ltd.(Defaulter)",
    "FANM - First AL-Noor Modarba", "FASM - Faisal Spinning Mills Ltd.", "FATIMA - Fatima Fertilizer Company Ltd.",
    "FCCL - Fauji Cement Company Ltd.", "FCEL - First Capital Equities Ltd.(Defaulter)",
    "FCEPL - Frieslandcampina Engro Pakistan Ltd.", "FCIBL - First Credit and Investment Bank Ltd.", "FCL - Fast Cables Ltd",
    "FCONM - First Constellation Modaraba", "FCSC - First Capital Sec. Corp. Ltd.", "FDMF - First Dawood Mutual Fund",
    "FDPL - First Dawood Properties Ltd (Formerly FDIBL)", "FECM - First Elite Capital Modaraba", "FECTC - Fecto Cement Ltd.",
    "FEM - First Equity Modarba", "FEROZ - Ferozsons Laboratories Ltd.", "FFBL - Fauji Fertilizer Bin Qasim Ltd. Consolidated",
    "FFC - Fauji Fertilizer Company Ltd. Consolidated", "FFL - Fauji Foods Ltd.", "FFLM - First Fidelity Leasing Modaraba",
    "FFLNV - Fauji Foods Non Voting", "FHAM - First Habib Modarba Ltd.", "FIBLM - First IBL Modaraba",
    "FIL - Fateh Industries Ltd.(Defaulter)", "FIM - First Investec Modaraba.(Defaulter)", "FIMM - First Imrooz Modaraba",
    "FLYNG - Flying Cement Ltd.", "FML - Feroze1888 Mills Ltd.", "FNBM - First National Bank Modaraba.(Defaulter)",
    "FNEL - First National Equities Ltd.", "FPJM - First Punjab Modarba", "FPRM - First Paramount Modaraba",
    "FRCL - Frontier Ceramics Ltd.", "FRSM - Faran Sugar Mills Ltd.", "FSWL - Fateh Sports Wear Ltd.(Defaulter)",
    "FTHM - Fateh Textile Mills Ltd.(Defaulter)", "FTMM - First Treet Mfg Modaraba", "FTSM - First Tri-Star Modarba.",
    "FUDLM - First UDL Modarba", "FZCM - Fazal Cloth Mills Ltd.", "GADT - Gadoon Textile Mills Ltd.",
    "GAIL - Ghani Automobile Industries Ltd. merged with GVGL", "GAL - Ghandhara Automobiles Ltd. Consolidated",
    "GAMON - Gammon Pakistan Ltd.", "GASF - Golden Arrow Selected Stocks Fund Ltd.", "GATI - Gatron Industries Ltd.",
    "GATM - Gul Ahmed Textile Mills Ltd.", "GCIL - Ghani Chemical Industries Ltd.(G3 Technologies)",
    "GEMBCEM - Burj Clean Energy Modaraba (Gem)", "GEMBLUEX - Blue-Ex Ltd. (Formerly Universal Network System",
    "GEMMEL - Mughal Energy Ltd (GEM)", "GEMPAPL - Pak Agro Packaging Ltd.(Gem Board)",
    "GEMSPNL - Supernet Ltd. Consolidated (Gem Board)", "GENP - Genertech Pakistan Ltd.",
    "GFIL - Ghazi Fabrics International Ltd.", "GGGL - Ghani Global Glass Ltd.", "GGL - Ghani Global Holdings Ltd. Consolidated",
    "GHGL - Ghani Glass Ltd.", "GHNI - Ghandhara Industries Ltd.", "GIL - Good Luck Industries Ltd.",
    "GLAT - Glomour Textile Mills Ltd.", "GLAXO - Glaxosmithkline (Pak) Ltd.", "GLOT - Globe Textile Mills Ltd.(Defaulter)",
    "GLPL - Gillette Pakistan Ltd.", "GOC - GOC Pakistan Ltd.", "GOEM - Globe Textile Mills (OE) Ltd.",
    "GRR - Globe Residency Reit", "GRYL - Grays Leasing Ltd.", "GSPM - Gulshan Spinning Mills Ltd.(Defaulter)",
    "GTYR - Ghandhara Tyre & Rubber Co. Ltd.", "GUSM - Gulistan Spinning Mills Ltd.(Defaulter)",
    "GUTM - Gulistan Textile Mills Ltd.(Defaulter)", "GVGL - Ghani Value Glass Ltd.", "GWLC - Gharibwal Cement Ltd.",
    "HABSM - Habib Sugar Mills Ltd.", "HADC - Haydary Construction Co. Ltd.(Defaulter)", "HAEL - Hala Enterpries Ltd.",
    "HAFL - Hafiz Ltd.", "HAJT - Hajra Textile Mills Ltd.(Defaulter)", "HALEON - Haleon Pakistan Ltd.(Formerly GSKCH Pak Ltd)",
    "HASCOL - Hascol Petroleum Ltd.(Defaulter)", "HATM - Hamid Textile Mills Ltd.(Defaulter)", "HBL - Habib Bank Ltd.",
    "HCAR - Honda Atlas Cars (Pakistan) Ltd.", "HGFA - HBL Growth Fund - (A)", "HICL - Habib Insurance Co. Ltd.",
    "HIFA - HBL Investment Fund - (A)", "HINO - HinoPak Motors Ltd.", "HINOON - Highnoon Laboratories Ltd.",
    "HIRAT - Hira Textile Mills Ltd.(Defaulter)", "HKKT - Hakkim Textile Mills Ltd.(Defaulter)",
    "HMB - Habib Metropolitan Bank Ltd.", "HMIM - Haji Mohammed Ismail Mills Ltd.(Defaulter).",
    "HMM - Habib Metro Modarba(Merger by 1st Habib Mod.)", "HPL - Hoechst Pakistan Ltd.(Sanofi-Aventis)",
    "HRPL - Habib Rice Products Ltd.", "HSPI - Huffaz Seamless Pipe Industries Ltd. (Defaulter)", "HTL - Hi-Tech Lubricants Ltd",
    "HUBC - The Hub Power Company Ltd. Consolidated", "HUMNL - Hum Network Ltd.", "HUSI - Husein Industries Ltd.",
    "HWQS - Haseeb Waqas Sugar Mills Ltd.(Defaulter)", "IBFL - Ibrahim Fibre Ltd.", "IBLHL - IBL HealthCare Ltd.",
    "ICCI - I.C.C. Industries Ltd.(Defaulter)", "ICIBL - Invest Capital Investment Bank Ltd.", "ICL - Ittehad Chemicals Ltd.",
    "IDRT - Idrees Textile Mills Ltd.", "IDSM - Ideal Spinning Mills Ltd.", "IDYM - Indus Dyeing Manufacturing Co. Ltd.",
    "IGIBL - IGI Investment Bank Ltd.", "IGIHL - IGI Holdings Ltd. Consolidated", "IGIL - IGI Life Insurance Ltd",
    "ILP - Interloop Ltd", "ILTM - Island Textile Mills Ltd.", "IMAGE - Image Pakistan Ltd.", "IML - Imperial Ltd.",
    "IMS - Intermarket Securities Ltd. FORMERLY: EFG Hermes )", "INDU - Indus Motor Company Ltd.",
    "INIL - International Industries Ltd. Consolidated", "INKL - International Knitwear Ltd.", "INl - 786 Investment Ltd.",
    "INMF - Investec Mutual Fund Ltd.(Defaulter)", "IPAK - International Packaging Films Ltd",
    "ISHT - Ishtiaq Textile Mills Ltd.", "ISIL - Ismail Industries Ltd.", "ISL - International Steels Ltd.",
    "ITSL - Investec Securities Ltd.", "ITTEFAQ - Ittefaq Iron Industries Ltd.", "JATM - J. A. Textile Mills Ltd.",
    "JDMT - Janana-de-Malucho Textile Mills Ltd.", "JDWS - JDW Sugar Mills Ltd.", "JGICL - Jubilee General Insurance Co. Ltd.",
    "JKSM - J. K. Spinning Mills Ltd.", "JLICL - Jubilee Life Insurance Co. Ltd.", "JOPP - Johnson & Phillips (Pak) Ltd.",
    "JOVC - Javed Omer Vohra & Co. Ltd.", "JPGL - Japan Power Generation Ltd.(Defaulter)", "JSBL - JS Bank Ltd.",
    "JSCL - Jahangir Siddiqui & Company Ltd.", "JSGCL - JS Global Capital Ltd.", "JSIL - JS Investment Ltd.",
    "JSML - Jauharabad Sugar Mills Ltd.", "JUBS - Jubilee Spinning and Weaving Mills Ltd.(Defaulter)",
    "JVDC - Javedan Corporation Ltd.", "JVDCPS - Javedan Corporation Preference", "KAKL - Kaiser Arts & Krafts Ltd.(Defaulter)",
    "KAPCO - Kot Addu Power Company Ltd.", "KASBM - KASB Modaraba", "KCL - Karam Ceramices Ltd.", "KCORP - KASB Corporation Ltd.",
    "KEL - K-Electric Ltd.", "KHTC - Khyber Tobacco Company Ltd.", "KHYT - Khyber Textile Mills Ltd.",
    "KML - Kohinoor Mills Ltd.", "KOHC - Kohat Cement Company Ltd.", "KOHE - Kohinoor Energy Ltd.",
    "KOHP - Kohinoor Power Company Ltd.", "KOHTM - Kohat Textile Mills Ltd.", "KOIL - Kohinoor Industries Ltd.",
    "KOSM - Kohinoor Spinning Mills Ltd.", "KPUS - Khairpur Sugar Mills Ltd.", "KSBP - KSB Pumps Company Ltd.",
    "KSTM - Khalid Siraj Textile Mills Ltd.(Defaulter)", "KTML - Kohinoor Textile Mills Ltd.",
    "LCI - Lucky Core Industries Ltd. I.C.I Pakistan Consolid", "LEUL - Leather Up Ltd.",
    "LIVEN - Liven Pharma Ltd. (Formerly LMSM)", "LOADS - Loads Ltd.", "LOTCHEM - Lotte Chemical (Pak) Ltd.",
    "LPGL - Leiner Pak Gelatine Ltd.", "LPL - Lalpir Power Ltd.", "LSECL - LSE Capital Ltd",
    "LSEFSL - LSE Financial Services Ltd", "LSEPL - LSE Proptech Ltd.", "LSEVL - LSE Ventures Ltd.",
    "LUCK - Lucky Cement Ltd. Consolidated", "MACFL - MACPAC Films Ltd.", "MACTER - Macter International Ltd.",
    "MARI - Mari Energies Ltd.", "MCB - MCB Bank Ltd.", "MCBIM - MCB Investment Management Ltd.(MCB-Arif Habib)",
    "MDTL - Media Times Ltd.", "MDTM - Mehr Dastgir Textile Mills Ltd.", "MEBL - Meezan Bank Ltd.",
    "MEHT - Mehmood Textile Mills Ltd.", "MERIT - Merit Packaging Ltd.", "META - MetaTech Trading Ltd. (Formerly MetaTech Health)",
    "MFFL - Mitchells Fruit Farms Ltd.", "MFL - Matco Foods Ltd. Consolidated", "MFTM - Muhammad Farooq Textile Mills Ltd.(Defaulter)",
    "MIRKS - Mirpurkhas Sugar Mills Ltd.", "MLCF - Maple Leaf Cement Factory Ltd. Consolidated", "MODAM - Modaraba Al - Mali",
    "MOHE - Mohib Exports Ltd.(Defaulter)", "MOIL - Morafco Industries Ltd.", "MQTM - Maqbool Textile Mills Ltd.",
    "MRNS - Mehran Sugar Mills Ltd.", "MSCL - Metropolitan Steel Corporation Ltd.", "MSOT - Masood Textile Mills Ltd.",
    "MSOTPS - Masood Textile Preference", "MTL - Millat Tractors Ltd. Consolidated", "MUBT - Mubarak Textile Mills Ltd.(Defaulter)",
    "MUGHAL - Mughal Iron & Steels Ind. Ltd.", "MUKT - Mukhtar Textile Mills Ltd.", "MUREB - Murree Brewery Company Ltd.",
    "MWMP - Mandviwala Mauser Plastic Industries.", "MZSM - Mirza Sugar Mills Ltd.", "NAFL - National Fibres Ltd.(Defaulter)",
    "NAGC - Nagina Cotton Mills Ltd.", "NATF - National Foods Ltd.", "NATM - Nadeem Textile Mills Ltd.",
    "NBP - National Bank Of Pakistan", "NCL - Nishat Chunian Ltd.", "NCML - Nazir Cotton Mills Ltd.(Defaulter)",
    "NCPL - Nishat Chunian Power Ltd.", "NESTLE - Nestle Pakistan Ltd.", "NETSOL - Netsol Technology Ltd.",
    "NEXT - Next Capital Ltd.", "NICL - Nimir Industrial Chemicals Ltd.", "NINA - Nina Industries Ltd.(Defaulter)",
    "NML - Nishat Mills Ltd. Consolidated", "NONS - Noon Sugar Mills Ltd.", "NORS - Noor Silk Mills Ltd.",
    "NPL - Nishat Power Ltd.", "NPSM - N. P. Spinning Mills Ltd.", "NRL - National Refinery Ltd.", "NRSL - Nimir Resins Ltd.",
    "NRSLR - Nimir Resins Ltd. (R)", "NSRM - National Silk and Rayon Mills Ltd.", "OBOY - Oilboy Energy Ltd.",
    "OCTOPUS - Octopus Digital Ltd", "OGDC - Oil & Gas Development Company Ltd.",
    "OLPL - OLP Financial Services Pakistan(Formerly ORIX Leas", "OLPM - OLP Modaraba(Formerly ORIX Modaraba",
    "OLTM - Olympia Textile Mills Ltd.", "OML - Olympia  Mills Ltd.", "ORM - Orient Rental Modaraba",
    "OTSU - Otsuka Pakistan Ltd.", "PABC - Pakistan Aluminium Beverage Cans Ltd.", "PACE - Pace Pakistan Ltd.",
    "PAEL - Pak Elektron Ltd.", "PAKCEM - Pakcem Ltd.", "PAKD - Pak Datacom Ltd.", "PAKL - Pak Leather Crafts Ltd.",
    "PAKMI - First Pak Modaraba. (Delisted)", "PAKOXY - Pakistan Oxygen Ltd.", "PAKRI - Pakistan Reinsurance Co. Ltd.",
    "PAKT - Pakistan Tobacco Company Ltd.", "PASL - Pervez Ahmed Consultansy Services Ltd.(Defaulter)",
    "PASM - Parmount Spinning Mills Ltd.(Defaulter)", "PCAL - Pakistan Cables Ltd.", "PCML - Punjab Cotton Mills Ltd.",
    "PDGH - Prudential Discourt & Guarantee House Ltd.(Default", "PECO - Pakistan Engineering Co. Ltd.(Non-Compliant)",
    "PELPS - Pak Elektron Preference", "PGCL - Pakistan Gum and Chemicals Ltd.", "PGIC - Pakistan Guarantee Insurance Co. Ltd.",
    "PGLC - Pak Gulf  Leasing  Co. Ltd.", "PHDL - Pakistan Hotels Developers Ltd.(This company is windup)",
    "PIAHCLA - PIA Holding Company Ltd. (A)", "PIAHCLB - PIA Holding Company Ltd B",
    "PIBTL - Pakistan International Bulk Terminal Ltd.", "PICL - Pak Industrial & Commercial Leasing Ltd.(Defaulter",
    "PICT - Pakistan International Container Terminal Ltd.", "PIL - PICIC Insurance Ltd.(Defaulter)",
    "PIM - Popular Islamic Modaraba", "PINL - Premier Insurance Co. Ltd.", "PIOC - Pioneer Cement Ltd.",
    "PKGI - Pakistan General Insurance Co. Ltd.", "PKGP - Pakgen Power Ltd.", "PKGS - Packages Ltd. Consolidated",
    "PMI - First Prudential Modaraba", "PMPK - Philip Morris (Pakistan) Ltd.",
    "PMRS - The Premier Sugar Mills and Distillery Co. Ltd.", "PNGRS - Pangrio Sugar Mills Ltd.",
    "PNSC - Pakistan National Shipping Corp. Consolidated", "POAF - Pak Oman Advantage Fund", "POL - Pakistan Oilfields Ltd.",
    "POML - Punjab Oil Mills Ltd.", "POWER - Power Cement Ltd.", "PPL - Pakistan Petroleum Ltd.",
    "PPP - Pakistan Paper Prouducts Ltd.", "PPVC - Pakistan PVC Ltd.(Defaulter)", "PREMA - At-Tahur Ltd.",
    "PRET - Premium Textile Mills Ltd.", "PRIB - Prudential Investment Bank Ltd.(Defaulter).",
    "PRIC - Progressive Insurance Co. Ltd.(Defaulter)", "PRL - Pakistan Refinery Ltd.", "PRWM - Prosperity Weaving Mills Ltd.",
    "PSEL - Pakistan Services Ltd.", "PSMC - Pak Suzuki Motor Company Ltd.", "PSO - Pakistan State Oil Company Ltd. Consolidated",
    "PSX - Pakistan Stock Exchange Ltd.", "PSYL - Pakistan Synthetic Ltd.", "PTC - Pakistan Telecommunication Co. Ltd.",
    "PTL - Panther Tyres Ltd.", "PUDF - Prudential Stocks Fund Ltd.(Defaulter)", "QUET - Quetta Textile Mills Ltd. (Defaulter)",
    "QUICE - Quice Food Industries Ltd.", "QUSW - Quality Steel Works Ltd.(Defaulter)",
    "RCML - Reliance Cotton Spinning Mills Ltd. Consolidated", "REDCO - Redco Textiles Ltd.", "REGAL - Regal Ceramics Ltd.(Defaulter)",
    "REST - Resham Textile Industries Ltd.", "REWM - Reliance Weaving Mills Ltd.", "RICL - Reliance  Insurance Co. Ltd.",
    "RMPL - Rafhan Maize Products Co. Ltd.", "RPL - Roshan Packages Ltd.", "RUBY - Ruby Textile Mills Ltd.(Defaulter)",
    "RUPL - Rupali Polyester Ltd.", "SAIF - Saif Textile Mills Ltd.", "SALT - Salfi Textile Mills Ltd.",
    "SANE - Salman Noman Enterpries Ltd.(Defaulter)", "SANSM - Sanghar Sugar Mills Ltd.", "SAPT - Sapphire Textile Mills Ltd.",
    "SARC - Sardar Chemical Industries Ltd.", "SASML - Sindh Abadgars Sugar Mills Ltd.", "SAZEW - Sazgar Engineering  Works Ltd.",
    "SBL - Samba Bank Ltd.", "SCBPL - Standard Chartered Bank Pakistan Ltd.", "SCHT - Schon Textiles Ltd.(Defaulter)",
    "SCL - Shield Corporation Ltd.", "SCLL - Standard Chartered Leasing Ltd.", "SDIL - Saleem Denim Industries Ltd.",
    "SDOT - Sadoon Textile Industries Ltd.(Defaulter)", "SEARL - The Searle Company Ltd. Consolidated",
    "SEL - Sitara Energy Ltd.", "SEPCO - Southern Electric Company Ltd.", "SEPL - Security Paper Ltd.",
    "SERT - Service Industries Textiles Ltd.", "SFAT - Safa Textiles Ltd.(Defaulter)", "SFL - Sapphire Fibers Ltd.",
    "SGABL - S.G. Allied Businesses Ltd. Defaulter", "SGF - Service Global Footwear Ltd.", "SGMLPS - Shakarganj (R.C.Pr)",
    "SGPL - S.G. Power Ltd.", "SHCI - Shaffi Chemical Ind. Ltd.(Defaulter)", "SHCM - Shadman Cotton Mills Ltd.",
    "SHDT - Shadab Textile Mills Ltd.", "SHEZ - Shezan International Ltd.", "SHFA - Shifa International Hospitals Ltd.",
    "SHJS - Shahtaj Sugar Mills Ltd.", "SHNI - Shaheen Insurance Co. Ltd.", "SHSML - Shahmurad Sugar Mills Ltd.",
    "SIBL - Security Investment Bank Ltd.", "SICL - Standard Insurance Company Ltd.(Defaulter)",
    "SIEM - Siemens Pakistan Engineering Co. Ltd.", "SILK - SilkBank Ltd.", "SILKR1 - Silkbank (R)", "SINDM - Sindh Modaraba",
    "SITC - Sitara Chemical Industries Ltd.", "SJTM - Sajjad Textile Mills Ltd.", "SKRS - Sakrand Sugar Mills Ltd. (Defaulter)",
    "SLCL - Security Leasing Corporation Ltd.(Defaulter)", "SLCPA - Sec.Lea(Pre)9.1%",
    "SLGL - Secure Logistics Group Ltd (Consolidated)", "SLL - SME Leasing Ltd.", "SLSO - Saleem Sugar Mills Ltd.",
    "SLSOPP - Saleem Suger Mills Ltd. (Pp)", "SLSOPVI - Saleem Sugar Mills Ltd. (P) 6%", "SLYT - Sally Textile Mills Ltd.(Defaulter)",
    "SMBLCPSA - Summit Bank Pref Class A", "SMBLCPSB - Summit Bank Pref Class B", "SMCPL - Safe Mix Concrete Ltd.",
    "SML - Shakerganj Ltd. (Defaulter)", "SNAI - Sana Industries Ltd.", "SNBL - Soneri Bank Ltd.",
    "SNGP - Sui Northern Gas Pipelines Ltd.", "SPCL - Saudi Pak Consultancy Co. Ltd. (Formerly SPLC)",
    "SPEL - Synthetic Products Enterprises Ltd.", "SPL - Sitara Peroxide Ltd. Defaulter", "SPWL - Saif Power Ltd.",
    "SRSM - Sargoda Spinning Mills Ltd.", "SRVI - Service Industries Ltd.", "SSGC - Sui Southern Gas Company Ltd.",
    "SSIC - Silver Star Insurance Company Ltd.(Defaulter)", "SSML - Saritow Spinning Mills Ltd. (Defaulter)",
    "SSOM - S. S. Oil Mills Ltd.", "STCL - Shabbir Tiles & Ceramics Ltd.", "STJT - Shahtaj Textile Mills Ltd.",
    "STL - Supernet Technologies Ltd. (Formerly Hallmark Co.)", "STML - Shams Textile Mills Ltd.",
    "STPL - Siddiqsons Tin Plate Ltd. (Defaulter)", "STYLERS - Stylers International Ltd", "SUCM - Sunshine Cotton Mills Ltd.",
    "SUHJ - Suhail Jute Mills Ltd.(Defaulter)", "SURAJ - Suraj Ghee Industries Ltd.(Defaulter)", "SURC - Suraj Cotton Mills Ltd.",
    "SUTM - Sunrays Textile Mills Ltd.", "SYM - Symmetry Group Ltd. Consolidated", "SYS - Systems Ltd.",
    "SZTM - Shahzad Textile Mills Ltd.", "TAJT - Taj Textile Mills Ltd.(Defaulter)",
    "TATM - Tata Textile Mills Ltd.Merger Island Salfi Textil", "TBL - Treet Battery Ltd", "TCLTC - Treet Corporation (Ptc)",
    "TCORP - Tariq Corporation Ltd.(Hussein Sugar)", "TELE - Telecard Ltd. Consolidated", "TGL - Tariq Glass Industries Ltd.",
    "THALL - Thal Ltd.", "THCCL - Thatta Cement Company Ltd.", "TICL - Thal Industries Corporation Ltd.",
    "TOMCL - The Organic Meat Company Ltd.", "TOWL - Towellers Ltd.", "TPL - TPL Corporation Ltd.", "TPLI - TPL Insurance Ltd.",
    "TPLL - TPL Life Insurance Ltd Formerly (Dar-es-Salaam Tex", "TPLP - TPL Properties Ltd.", "TPLRF1 - TPL REIT Fund I",
    "TPLT - TPL Trakker Ltd", "TREET - Treet Corporation Ltd. Consolidated", "TREI - Transmission Engineering Ind. Ltd.",
    "TRG - TRG Pakistan Ltd.", "TRIBL - Trust Investment Bank Ltd.(Defaulter)", "TRIPF - Tri-Pack Films Ltd.",
    "TRSM - Trust Modarba", "TSBL - Trust Securities and  Brokerage Ltd.", "TSMF - Tri-Star Mutual Fund Ltd.",
    "TSML - Tandlianwala Sugar Mills Ltd.", "TSPL - Tri-Star Power Ltd.", "UBDL - United Brands Ltd.",
    "UBL - United Bank Ltd.", "UCAPM - UNICAP Modarba", "UDLI - UDL International Ltd",
    "UDPL - United Distributors Pakistan Ltd.", "UNIC - United Insurance Co. of Pakistan Ltd.", "UNITY - Unity Foods Ltd.",
    "UPFL - Unilever Pakistan Foods Ltd.", "USMT - Usman Textile Mills Ltd.(Defaulter)", "UVIC - Universal Insurance Company Ltd.",
    "WAFI - Wafi Energy Pakistan Ltd (Shell Pakistan Ltd)", "WAHN - Wah-Nobel Chemicals Ltd.",
    "WAVES - Waves Corporation Ltd. Consolidated", "WAVESAPP - Waves Home Appliances Ltd. (Formerly Samin Textile",
    "WTL - WorldCall Telecom Ltd.", "WYETH - Wyeth Pakistan Ltd. (Delisted)", "YOUW - Yousuf Weaving Mills Ltd.",
    "ZAHID - Zahidjee Textile Mills Ltd.", "ZAL - Zarea Ltd", "ZELP - Zeal Pak Cement Factory Ltd.(Defaulter)",
    "ZHCM - Zahur Cotton Mills Ltd.(Defaulter)", "ZIL - ZIL Ltd.", "ZTL - Zephyr Textile Ltd."
]
print(f"--- Loaded {len(availableTags)} tags ---")


# 3. Create ipywidgets
search_box = widgets.Text(
    placeholder='Type to filter stocks...',
    description='Filter:',
    disabled=False,
    layout={'width': '400px'}
)

dropdown = widgets.Dropdown(
    options=availableTags,
    description='Select Stock:',
    disabled=False,
    layout={'width': '500px'},
    value=None # Important: Start with no value selected
)

# 4. Function to filter dropdown
def update_dropdown_options(change):
    query = change['new'].strip().upper()
    current_val = dropdown.value

    if not query:
        new_options = availableTags
    else:
        new_options = []
        for item in availableTags:
            item_upper = item.upper()
            parts = item.split(" - ", 1)
            symbol = parts[0].strip().upper() if parts else ""
            # Match start of symbol OR anywhere in full text
            if (symbol and symbol.startswith(query)) or (query in item_upper):
                new_options.append(item)

    # Update options, try to preserve selection
    dropdown.options = new_options
    if current_val in new_options:
        dropdown.value = current_val
    elif not new_options:
        dropdown.value = None

# 5. Link search box to dropdown update function
search_box.observe(update_dropdown_options, names='value')

# 6. Display widgets - USER INTERACTION REQUIRED HERE
print("\n--->>> PLEASE SELECT A STOCK FROM THE DROPDOWN BELOW <<<---")
display(widgets.VBox([search_box, dropdown]))
print("--- After selecting, run the next cell to start scraping ---")

In [ ]:
# --- Cell 2: Scraping Execution ---
# >>> RUN THIS CELL *AFTER* SELECTING A STOCK IN CELL 1 <<<

# 1. Get Selected Company from Widget
selected_company_full_text = dropdown.value

# 2. --- VALIDATION ---
if not selected_company_full_text or selected_company_full_text not in availableTags:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("ERROR: No valid stock selected in the dropdown above.")
    print("Please re-run Cell 1, select a stock, then run this cell again.")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
else:
    # 3. Parse Selection
    try:
        parts = selected_company_full_text.split(" - ", 1)
        company_search_term = parts[0].strip() # SYMBOL for send_keys
        # Use the exact selection for clicking later
        company_to_select_text = selected_company_full_text
        print(f"--- Preparing to scrape for: ---")
        print(f"   Symbol (for typing): '{company_search_term}'")
        print(f"   Full Text (for clicking): '{company_to_select_text}'") # Renamed variable
    except Exception as e:
        print(f"Error parsing selected company '{selected_company_full_text}': {e}")
        company_search_term = None
        company_to_select_text = None # Clear this too

    # 4. --- SET FIXED DATES ---
    start_date_obj = date(2023, 12, 4)
    end_date_obj = date.today()
    # Format dates as MM/DD/YYYY for the website input
    start_date_str = start_date_obj.strftime('%m/%d/%Y')
    end_date_str = end_date_obj.strftime('%m/%d/%Y')
    print(f"   Start Date: {start_date_str}")
    print(f"   End Date: {end_date_str}")
    print("-" * 30)

    # 5. --- Proceed only if company was parsed successfully ---
    if company_search_term and company_to_select_text:

        # --- Setup Chrome Options ---
        chrome_options = Options()
        args = [
            "--headless", "--no-sandbox", "--disable-dev-shm-usage",
            "--window-size=1920,1080", "--disable-gpu", "--disable-extensions",
            "--proxy-server='direct://'", "--proxy-bypass-list=*", "--start-maximized",
            '--log-level=3'
        ]
        for arg in args: chrome_options.add_argument(arg)
        chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
        temp_user_data_dir = None
        driver = None
        all_data_list = []
        header = []
        # --- Initialize variables before the main try block ---
        no_records_found_flag = False # <<< FIX 2: Initialize here
        header_extracted = False      # <<< Good Practice: Initialize here too

        # --- Start of original scraping logic ---
        try:
            temp_user_data_dir = tempfile.mkdtemp()
            chrome_options.add_argument(f"--user-data-dir={temp_user_data_dir}")
            print(f"Using temporary user data dir: {temp_user_data_dir}")

            # --- Initialize WebDriver ---
            try:
                # Optional: Explicitly set Service if webdriver-manager fails sometimes
                # from webdriver_manager.chrome import ChromeDriverManager
                # service = Service(ChromeDriverManager().install())
                # driver = webdriver.Chrome(service=service, options=chrome_options)
                driver = webdriver.Chrome(options=chrome_options) # Keep original if it works
            except Exception as driver_err:
                print(f"ERROR initializing WebDriver: {driver_err}")
                raise
            print("WebDriver initialized.")
            # --- END Initialize WebDriver ---

            def adjust_scraped_date(date_str): # Keep date adjustment function local
                try:
                    date_str = date_str.strip().replace(" ", "")
                    if not date_str: return ""
                    parsed_date = None
                    formats_to_try = ["%d-%b-%y", "%d-%b-%Y", "%d%b%y", "%d%b%Y"]
                    for fmt in formats_to_try:
                        try:
                            parsed_date = datetime.strptime(date_str, fmt)
                            break
                        except ValueError: continue
                    if parsed_date is None: raise ValueError(f"Unparseable date: {date_str}")
                    if parsed_date.year < 100: # Handle 2-digit year
                        if parsed_date.year >= 70: parsed_date = parsed_date.replace(year=parsed_date.year + 1900)
                        else: parsed_date = parsed_date.replace(year=parsed_date.year + 2000)
                    settlement_date = parsed_date + timedelta(days=1)
                    while settlement_date.weekday() >= 5: settlement_date += timedelta(days=1)
                    return settlement_date.strftime("%d-%b-%Y").upper()
                except Exception as e:
                    print(f"Date conversion failed for '{date_str}'. Error: {str(e)}")
                    return date_str # Return original if conversion fails

            url = "http://www.scstrade.com/MarketStatistics/MS_HistoricalPrices.aspx"
            print(f"Navigating to {url}")
            driver.get(url)
            print("Page loaded.")

            # Use wait times similar to your original code
            wait_timeout = 20
            long_wait_timeout = 75 # Define timeout value
            wait = WebDriverWait(driver, wait_timeout)
            long_wait = WebDriverWait(driver, long_wait_timeout)

            print("Locating input fields...")
            company_name_input = wait.until(EC.presence_of_element_located((By.ID, "tags")))
            start_date_input = driver.find_element(By.ID, "date1")
            end_date_input = driver.find_element(By.ID, "date2")
            view_price_button_locator = (By.ID, "btn1")
            print("Input fields located.")

            # --- Handle Autocomplete Selection ---
            # *** USE VARIABLES DERIVED FROM WIDGET ***
            print(f"Entering Company Search Term: {company_search_term}")
            company_name_input.clear()
            company_name_input.send_keys(company_search_term)
            time.sleep(1) # Pause

            autocomplete_dropdown_locator = (By.ID, "ui-id-1")
            # Use the FULL TEXT variable for clicking the option
            safe_company_text_click = company_to_select_text.replace("'", "\\'").replace('"', '\\"')
            # Try specific div match first
            company_option_xpath = f"//ul[@id='ui-id-1']/li/div[normalize-space(.)='{safe_company_text_click}']"
            company_option_locator = (By.XPATH, company_option_xpath)

            try:
                print(f"Waiting for autocomplete option: '{company_to_select_text}'")
                wait.until(EC.visibility_of_element_located(autocomplete_dropdown_locator))
                option_element = wait.until(EC.element_to_be_clickable(company_option_locator))
                print("Autocomplete option found. Clicking...")
                driver.execute_script("arguments[0].click();", option_element) # JS click
                print(f"Clicked '{company_to_select_text}'.")
                time.sleep(1) # Pause
            except TimeoutException:
                print(f"FATAL: Could not find or click '{company_to_select_text}' in autocomplete.")
                raise
            # --- END Autocomplete ---

            # --- USE DERIVED DATES ---
            print(f"Entering Start Date: {start_date_str}")
            driver.execute_script(f"arguments[0].value = '{start_date_str}';", start_date_input)
            time.sleep(0.5)

            print(f"Entering End Date: {end_date_str}")
            driver.execute_script(f"arguments[0].value = '{end_date_str}';", end_date_input)
            time.sleep(0.5)

            print("Waiting for 'View Price' button...")
            view_price_button_element = wait.until(EC.element_to_be_clickable(view_price_button_locator))

            print("Attempting click on 'View Price' button...")
            try:
                driver.execute_script("arguments[0].click();", view_price_button_element) # JS click
                print("JS click successful.")
                print("Pausing for request initiation...")
                time.sleep(4) # Original was 2, slightly increased
            except Exception as e:
                 print(f"FATAL: Error during View Price click: {e}")
                 raise

            # --- Wait for the *first* page of results to load (Simplified) ---
            results_table_locator = (By.ID, "list")
            data_rows_locator = (By.CSS_SELECTOR, "#list > tbody > tr.jqgrow:not(.jqgrid-dummy-row)") # Exclude dummy
            no_records_locator = (By.XPATH, "//div[@id='pager_right']//div[contains(text(), 'No records to view')]")
            loading_indicator_locator = (By.ID, "load_list")
            header_row_locator = (By.CSS_SELECTOR, "#gview_list .ui-jqgrid-htable tr.ui-jqgrid-labels")
            header_cell_text_locator = (By.CSS_SELECTOR, "th div.ui-th-div")
            first_header_cell_text_locator = (By.CSS_SELECTOR, "#gview_list .ui-jqgrid-htable th:first-child div.ui-th-div")


            # *** SIMPLIFIED WAIT: Wait for loading indicator to disappear ***
            # *** AND THEN add a fixed sleep. This is less robust but closer ***
            # *** to original logic if explicit waits for data caused issues. ***
            print(f"Waiting for loading indicator to disappear... (Timeout: {long_wait_timeout}s)") # <<< FIX 1: Use defined variable
            long_wait.until(EC.invisibility_of_element_located(loading_indicator_locator))
            print("Loading indicator disappeared.")
            print("Pausing fixed time for data grid to potentially load...")
            time.sleep(4) # **** ADJUST THIS FIXED SLEEP TIME **** (Try 3-6 seconds)


            # --- Check for "No records" AFTER the pause ---
            # no_records_found_flag = False # <<< MOVED INITIALIZATION EARLIER
            try:
                 driver.find_element(*no_records_locator)
                 print("Initial search returned 'No records to view'. Exiting data processing.")
                 no_records_found_flag = True # Set the flag
            except NoSuchElementException:
                 print("Initial data potentially found. Proceeding to scrape.")
                 # --- Extract Header ---
                 # header_extracted = False # <<< MOVED INITIALIZATION EARLIER
                 try:
                    print("Waiting for header row...")
                    # Use a shorter wait for header as it should appear quickly if data is there
                    header_row_element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located(header_row_locator))
                    print("Header row visible. Extracting header...")
                    header_cells = header_row_element.find_elements(*header_cell_text_locator)
                    header = [cell.text.strip() for cell in header_cells if cell.text.strip()]
                    print("\nHeader:", header)
                    if header:
                        header_extracted = True
                        if len(header) != 7: print(f"WARN: Expected 7 headers, got {len(header)}.")
                    else:
                        print("ERROR: Header extraction resulted in empty list.")
                        header_extracted = False # Ensure it's false if extraction failed

                 except Exception as head_ex:
                    print(f"ERROR during header extraction: {head_ex}")
                    header_extracted = False # Ensure it's false on exception

                 # --- Pagination Loop (only if header OK) ---
                 if header_extracted:
                     page_num = 1
                     while True:
                         print(f"--- Scraping Page {page_num} ---")
                         time.sleep(1) # Pause before finding rows
                         try:
                             # Check if table body exists first
                             wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#list > tbody")))
                             current_rows = driver.find_elements(*data_rows_locator)
                         except Exception as find_err:
                             print(f"Error finding rows on page {page_num}: {find_err}")
                             break
                         print(f"Found {len(current_rows)} rows on page {page_num}.")

                         if not current_rows and page_num == 1:
                             # Double check if "No records" appeared *after* initial load
                             try:
                                driver.find_element(*no_records_locator)
                                print("Found 'No records' message after confirming initial data. Likely an async issue or filter change.")
                                no_records_found_flag = True
                             except NoSuchElementException:
                                print("WARNING: No rows found on page 1 even after waiting, and 'No records' not visible.")
                             break # Exit loop if no rows on page 1
                         if not current_rows and page_num > 1:
                             print("No more rows found, assuming end of pagination.")
                             break

                         rows_processed_this_page = 0
                         for row_index, row in enumerate(current_rows):
                             row_data = None
                             try:
                                 # Check visibility again, might become stale between find and access
                                 if not row.is_displayed():
                                     # print(f"Debug: Row {row_index+1} not displayed.")
                                     continue
                                 cols = row.find_elements(By.TAG_NAME, "td")
                                 row_data = [col.text.strip() for col in cols]
                                 # print(f"Debug: Row {row_index+1} data: {row_data}") # Optional debug line
                             except StaleElementReferenceException:
                                 print(f"WARN: Row {row_index+1} stale page {page_num}. Trying to re-find elements.")
                                 # A brief pause might help if the DOM is actively changing
                                 time.sleep(0.5)
                                 # Re-find rows on the current page and retry this specific index
                                 try:
                                     current_rows_refreshed = driver.find_elements(*data_rows_locator)
                                     if row_index < len(current_rows_refreshed):
                                         refreshed_row = current_rows_refreshed[row_index]
                                         if refreshed_row.is_displayed():
                                             cols = refreshed_row.find_elements(By.TAG_NAME, "td")
                                             row_data = [col.text.strip() for col in cols]
                                             print(f"   Successfully re-processed stale row {row_index+1}.")
                                         else:
                                             print(f"   Stale row {row_index+1} re-found but not displayed.")
                                             continue
                                     else:
                                         print(f"   Could not re-find stale row {row_index+1} by index.")
                                         break # Stop processing this page if re-find fails badly
                                 except Exception as refresh_err:
                                      print(f"ERROR: Failed attempt to handle staleness for row {row_index+1}: {refresh_err}")
                                      # Decide whether to continue with next row or break
                                      continue # Try next row
                             except Exception as row_err:
                                  print(f"ERROR processing row {row_index+1} on page {page_num}: {row_err}")
                                  continue # Skip this row

                             # Append data only if successfully processed
                             if row_data:
                                 if len(row_data) == len(header):
                                     row_dict = dict(zip(header, row_data))
                                     if "DATE" in row_dict:
                                         original_date = row_dict["DATE"]
                                         # Only adjust if date looks valid
                                         if original_date and original_date != '-':
                                             row_dict["DATE"] = adjust_scraped_date(original_date)
                                         else:
                                             print(f"WARN: Invalid or missing date in row {row_index+1}: '{original_date}'")
                                             row_dict["DATE"] = "" # Or keep original
                                     all_data_list.append(row_dict)
                                     rows_processed_this_page += 1
                                 else:
                                     print(f"WARN: Row {row_index+1} data/header mismatch. H:{len(header)}, R:{len(row_data)}. Data: {row_data}")
                         # End row processing loop for the page

                         # --- Check for Next Page ---
                         next_button_locator = (By.ID, "next_pager")
                         try:
                             time.sleep(0.8) # Allow UI to settle
                             # Ensure button exists before checking class
                             next_button = wait.until(EC.presence_of_element_located(next_button_locator))
                             button_classes = next_button.get_attribute("class") or "" # Handle None case

                             if "ui-state-disabled" in button_classes or "ui-disabled" in button_classes:
                                 print("Next button is disabled. Reached the last page.")
                                 break
                             else:
                                 print("Clicking Next Page button...")
                                 try:
                                     # Scroll into view just in case it's needed
                                     driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                                     time.sleep(0.4)
                                     # Try JS click first as it's often more reliable
                                     driver.execute_script("arguments[0].click();", next_button)
                                 except ElementClickInterceptedException:
                                      print("JS click intercepted, trying standard Selenium click...")
                                      wait.until(EC.element_to_be_clickable(next_button_locator)).click()
                                 except Exception as click_err:
                                      print(f"WARN: Click failed ({click_err}), trying standard click as fallback...")
                                      try:
                                          wait.until(EC.element_to_be_clickable(next_button_locator)).click()
                                      except Exception as final_click_err:
                                          print(f"FATAL: Final click attempt failed: {final_click_err}. Stopping pagination.")
                                          break # Stop pagination if click fails repeatedly

                                 print("Waiting for next page load...")
                                 # Wait for loading indicator to appear briefly (optional, confirms action)
                                 try:
                                     WebDriverWait(driver, 7).until(EC.visibility_of_element_located(loading_indicator_locator))
                                 except TimeoutException:
                                     print("Note: Loading indicator didn't appear immediately after click.")
                                     pass # It might be too fast or already gone

                                 # Crucial: Wait for loading indicator to disappear
                                 long_wait.until(EC.invisibility_of_element_located(loading_indicator_locator))
                                 print(f"Next page ({page_num + 1}) should be loaded.")
                                 page_num += 1
                                 time.sleep(2) # Pause after page load to allow elements to render

                         except TimeoutException:
                             print(f"Timed out waiting for Next button presence/state on page {page_num}. Assuming end.")
                             break
                         except Exception as page_e:
                             print(f"Error during pagination logic on page {page_num}: {page_e}")
                             break # Stop pagination on unexpected errors
                     # --- End While True (Pagination) ---
                 else: # Header failed
                      print("Skipping data scraping and pagination due to header extraction issue.")


        # --- Main Try block ends ---
        except Exception as e:
            print(f"\n--- An error occurred during scraping ---")
            print(f"Error type: {type(e).__name__}")
            print(f"Error message: {e}")
            import traceback
            traceback.print_exc()
            try: # Screenshot attempt
                if driver:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    screenshot_path = f"error_screenshot_{timestamp}.png"
                    driver.save_screenshot(screenshot_path)
                    print(f"Screenshot saved to {screenshot_path}")
            except Exception as screen_err:
                print(f"Could not save error screenshot: {screen_err}")

        finally:
            # --- Cleanup ---
            print("\n--- Cleaning up ---")
            if driver:
                print("Closing browser...")
                try:
                    driver.quit()
                    print("Browser closed.")
                except Exception as quit_err: print(f"Error closing browser: {quit_err}")
            if temp_user_data_dir and os.path.exists(temp_user_data_dir):
                try:
                    print(f"Removing temporary dir: {temp_user_data_dir}")
                    shutil.rmtree(temp_user_data_dir, ignore_errors=True)
                    if not os.path.exists(temp_user_data_dir): print("Temp dir removed.")
                    else: print("WARN: Temp dir might still exist.")
                except Exception as cleanup_error: print(f"Error cleaning temp dir: {cleanup_error}")

        # --- Post-processing: Save all data to CSV ---
        print("\n--- Saving results ---")
        # Use the flags initialized before the try block
        if all_data_list and header_extracted:
            safe_start_date_fn = start_date_obj.strftime('%d%b%Y')
            safe_end_date_fn = end_date_obj.strftime('%d%b%Y')
            # Sanitize company name for filename
            safe_company_name_fn = "".join(c for c in company_to_select_text if c.isalnum() or c in (' ', '_', '-')).strip().replace(' ', '_')
            output_filename = f"{safe_company_name_fn}_history_{safe_start_date_fn}_to_{safe_end_date_fn}.csv"
            print(f"Saving {len(all_data_list)} records to {output_filename}...")
            try:
                with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
                    # Use the extracted header as fieldnames
                    writer = csv.DictWriter(csvfile, fieldnames=header)
                    writer.writeheader()
                    writer.writerows(all_data_list)
                print(f"✅ Data saved successfully to {output_filename}")
                print("   (Refresh Colab file browser on the left to see it)")
            except Exception as csv_err: print(f"❌ Error saving data to CSV: {csv_err}")
        elif not all_data_list and no_records_found_flag:
             print("ℹ️ No data found for the selected criteria ('No records' message shown). CSV not created.")
        elif not all_data_list and not header_extracted:
             print("ℹ️ No data rows scraped and header extraction failed. CSV not created.")
        elif not all_data_list:
             print("ℹ️ No data rows were scraped (and 'No records' message was not detected). CSV file not created.")
        elif all_data_list and not header_extracted:
             print("⚠️ WARNING: Data scraped, but header extraction failed. Cannot save CSV correctly.")
        else: # Should not happen if logic above is correct, but as a fallback
             print("ℹ️ Scraping process ended. Check logs above. CSV not created.")
        print("="*50) # End separator

# --- End of Cell 2 ---